In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
ruta ="/home/donsson/proyectos/API/ventashistoricas52semanas.parquet"
df = pd.read_parquet(ruta)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199320 entries, 0 to 199319
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   origin          199320 non-null  string        
 1   price_subtotal  199320 non-null  float64       
 2   quantity        199320 non-null  float64       
 3   product_name    199320 non-null  object        
 4   invoice_name    199320 non-null  object        
 5   date_invoice    199320 non-null  datetime64[ns]
 6   store_name      199320 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3), string(1)
memory usage: 10.6+ MB


In [3]:
# Referncia de producto
df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")
df['ref_prod'] = df['product_name'].apply(lambda x: re.search(r'\(([^)]+)\)$', x).group(1) if pd.notna(x) and re.search(r'\(([^)]+)\)$', x) else None)
df['donsson'] = df['product_ref'].str.endswith('025')


df_d = df[df["donsson"]==True]

df_d = df_d.copy()


df_d["date"] = pd.to_datetime(df_d["date_invoice"])
df_d["año"] = df_d["date"].dt.year
df_d["mes"] = df_d["date"].dt.month


df_pivot = df_d.copy()
df_pivot = pd.pivot_table(df_d , values="quantity", index=["product_name","ref_prod","product_ref"] , columns=["año","mes"] , aggfunc= np.sum , fill_value= 0)
df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot['Promedio'] = df_pivot.mean(axis=1)


# 1. Sumar a nivel de línea para obtener el total
df_pivot['Total'] = df_pivot.iloc[:, 1:].sum(axis=1)

# 2. Ordenar el DataFrame por el total de ventas de forma descendente
df_pivot = df_pivot.sort_values(by='Total', ascending=False)

# 3. Calcular el porcentaje acumulado
df_pivot['acum%'] = df_pivot['Total'].cumsum() / df_pivot['Total'].sum()

# 4. Asignar la clasificación de Pareto
def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

df_pivot["Clasificacion"] = df_pivot["acum%"].apply(clasificar_pareto)

df_pivot = df_pivot.reset_index()

/tmp/ipykernel_290962/2054811732.py:18: FutureWarning: The provided callable <function sum at 0x7846e23d1800> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_pivot = pd.pivot_table(df_d , values="quantity", index=["product_name","ref_prod","product_ref"] , columns=["año","mes"] , aggfunc= np.sum , fill_value= 0)


In [4]:
df_pivot.head(10)

año                                       product_name    ref_prod  \
mes                                                                  
0    [DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...  025 DA2570   
1    [DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITAC...  025 DA2772   
2    [DAB02788025] DA2788  FILTRO AIRE DONSSON - Ch...  025 DA2788   
3    [DAB04570025] DA4570 FILTRO AIRE 2. DONSSON - ...  025 DA4570   
4    [DAB02666025] DA2666 FILTRO AIRE 1_ DONSSON -B...  025 DA2666   
5    [DAB02671025] DA2671 FILTROAIRE KUBOTA, NISSAN...  025 DA2671   
6    [DAB02968025] DA2968 FILTRO AIRE- HINO  (025 D...  025 DA2968   
7    [DAB08168025] DA8168 FILTRO AIRE DONSSON - FOT...  025 DA8168   
8    [DAB04666025] DA4666 FILTRO AIRE 2o. DONSSON B...  025 DA4666   
9    [DAB04772025] DA4772 FILTRO AIRE INTERNO BOBCA...  025 DA4772   

año  product_ref   2024                           2025                  \
mes                   9      10     11      12       1       2       3   
0    DAB02570025   39.0  2806.0  966.0  1019.0  1365.0  1351.0  1485.0   
1    DAB02772025   38.0  1557.0  492.0   973.0   660.0   564.0  1206.0   
2    DAB02788025  153.0  1265.0  793.0  1034.0   755.0   855.0   884.0   
3    DAB04570025   47.0  2139.0  961.0   858.0  1107.0  1422.0  1131.0   
4    DAB02666025   50.0  1121.0  386.0   560.0   855.0   793.0   831.0   
5    DAB02671025   25.0  1045.0  439.0   690.0   513.0   425.0   638.0   
6    DAB02968025   49.0   927.0  503.0   596.0   299.0   475.0   388.0   
7    DAB08168025   33.0   457.0  334.0   313.0   272.0   406.0   346.0   
8    DAB04666025   32.0   600.0  326.0   397.0   592.0   391.0   424.0   
9    DAB04772025   24.0   826.0  425.0   427.0   421.0   398.0   448.0   

año                                                         Total  \
mes       4       5       6       7       8       9                 
0    1616.0  1847.0  1106.0  1094.0  2083.0  1021.0  38099.571429   
1    1093.0  1098.0   799.0   696.0  2419.0   629.0  26156.285714   
2    1301.0  1048.0   982.0  1051.0   845.0  1174.0  25861.285714   
3     961.0  1335.0   728.0   685.0   144.0    41.0  24722.285714   
4    1069.0   952.0   551.0   674.0  1019.0   479.0  19964.285714   
5     351.0  1007.0   513.0   362.0   663.0   373.0  15069.285714   
6     357.0   493.0   294.0   455.0   456.0   497.0  12356.000000   
7     474.0   421.0   372.0   347.0   453.0   393.0   9869.142857   
8     633.0   439.0   341.0   192.0   173.0    40.0   9782.285714   
9     368.0   639.0   320.0   116.0    70.0     9.0   9599.571429   

año     Promedio     acum% Clasificacion  
mes                                       
0    2542.571429  0.061646           AAA  
1    1746.285714  0.103967           AAA  
2    1734.285714  0.145811           AAA  
3    1651.285714  0.185811           AAA  
4    1334.285714  0.218114           AAA  
5    1006.285714  0.242496           AAA  
6     827.000000  0.262488           AAA  
7     660.142857  0.278457           AAA  
8     654.285714  0.294284           AAA  
9     641.571429  0.309817           AAA

In [5]:
df_pivot.groupby("Clasificacion")["Clasificacion"].count()

Clasificacion
A      100
AAA     29
B      186
C      421
Name: Clasificacion, dtype: int64

In [7]:
df_pivot[df_pivot["product_ref"]=="DAB22690025"]

Empty DataFrame
Columns: [(product_name, ), (ref_prod, ), (product_ref, ), (2024, 9), (2024, 10), (2024, 11), (2024, 12), (2025, 1), (2025, 2), (2025, 3), (2025, 4), (2025, 5), (2025, 6), (2025, 7), (2025, 8), (2025, 9), (Total, ), (Promedio, ), (acum%, ), (Clasificacion, )]
Index: []

In [6]:
df_pivot.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx")